In [1]:
import os
import numpy as np
import pandas as pd
import random


# Set the print options
np.set_printoptions(suppress=True)


In [2]:
samples = np.genfromtxt('data/samples.csv', delimiter=",").astype(bool)
holdouts = np.genfromtxt('data/holdouts.csv', delimiter=",").astype(bool)
tests = np.genfromtxt('data/tests.csv', delimiter=",").astype(bool)
words = pd.read_csv('data/kidwords/kidwords.csv', names = ["word"])

In [4]:
# Specify the directory - used for several chunks below
directory = 'outputs/brute_force_2_with_frequency/'

# Tabulate dichotomous accuracy per model and place in a file
Extract the accuracy data for each sample and write to file. This takes a minute because it requires iterating through all the written accuracy files. Note that the accuracy summary datafile writes the accuracies out in a random order over rows, so you might need to arrange the rows if you want them to be in an orderly fashion.

This routine was copied from elsewhere and needs to be inspected. The code doesn't look right to me.

In [4]:
# Prepare an empty list to store the results
results = []

# Specify the indices

# Iterate over every file in the specified directory
for filename in os.listdir(directory):
    if filename.endswith("accuracies.csv"):
        # Extract the ID and level from the filename
        id_start = filename.find("sample_") + len("sample_")
        id_end = filename.find("hidden")
        ID = int(filename[id_start:id_end].replace("_", ""))

        level_start = filename.find("hidden_") + len("hidden_")
        level_end = filename.find("accuracies")
        level = int(filename[level_start:level_end].replace("_", ""))

        # Read the csv file
        data = np.genfromtxt(os.path.join(directory, filename), delimiter=',')

        # Tabulate the number of values equal to one and less than one
        equal_to_one = np.sum(data == 1)
        less_than_one = np.sum(data < 1)

        # Tabulate the values for each set of indices
        # if you want to determine
        train = np.sum(data[samples[:,ID]] == 1)/sum(samples[:,ID])
        test = np.sum(data[tests[:,ID]] == 1)/sum(tests[:,ID])
        holdout = np.sum(data[holdouts[:,ID]] == 1)/sum(holdouts[:,ID])

        # Save the results
        results.append([ID, level, train, test, holdout])

# Convert the results to a numpy array and write to a csv file
results_array = np.array(results)


pd.DataFrame(results_array).to_csv('outputs/brute_force_2_with_frequency/tabulated_dichotomous_accuracy.csv', index = False, header = ["model_id", "hidden", "train", "test", "holdout"])


## Full dataset for MSE
This will generate a file with all the right data, but is quite large (doesn't subset by sampling). It allow you to identify data for certain values of hidden unit.

In [5]:
filenames = []
for filename in os.listdir(directory):
    if filename.startswith("sample_"):
        if "error" in filename:
            if "hidden_20" in filename or "hidden_30" in filename or "hidden_40" in filename:
                filenames.append(filename)  

dfs = []

for filename in filenames:

    id_start = filename.find("sample_") + len("sample_")
    id_end = filename.find("hidden")
    ID = int(filename[id_start:id_end].replace("_", ""))

    level_start = filename.find("hidden_") + len("hidden_")
    level_end = filename.find("error")
    level = int(filename[level_start:level_end].replace("_", ""))

    one_file = pd.read_csv(f'outputs/brute_force_2_with_frequency/{filename}', names = ["mse"])
    one_file['word'] = words['word']
    one_file['model_id'] = ID
    one_file['hidden'] = level
    one_file['train'] = samples[:,ID]
    one_file['test'] = tests[:,ID]
    one_file['holdout'] = holdouts[:,ID]
    dfs.append(one_file)

df = pd.concat(dfs, ignore_index=True)

# this function needs to change if you are calculating accuracy from featurewise accuracy rather than from mse:
df['accuracy'] = df['mse'].apply(lambda x: 0 if x > 0 else 1)

In [6]:
df.shape

(86070000, 8)

### Write to file

In [ ]:
df = df[df['test'] == False]
df = df[['model_id', 'hidden', 'word', 'mse', 'train', 'holdout']]

# if you have generated a subset:
df.to_csv('outputs/brute_force_2_with_frequency/data_for_lmem_full.csv', index=False)


KeyError: 'test'

# LMEM subset data
Generate a random sample of 200 model IDs to make the subset data for the LMEM (which DK is running)

Write the MSE (files labeled with "error") data for the LMEM subset (the LMEMs can't be run on the entire dataset). This routine does look correct.

This chunk helps subset to create data for an LMEM, if you need to...

In [4]:
random.seed(345)

N = 1000

numbers = [str(i) for i in range(10000)]
subset = random.sample(numbers, N)

filenames = []
for filename in os.listdir(directory):
    for ID in subset:
        if filename.startswith("sample_"+ID+"_"):
            if "error" in filename:
              if filename or "hidden_20" in filename or "hidden_30" in filename or "hidden_40" in filename:
                  filenames.append(filename)  

In [5]:
dfs = []

for filename in filenames:

    id_start = filename.find("sample_") + len("sample_")
    id_end = filename.find("hidden")
    ID = int(filename[id_start:id_end].replace("_", ""))

    level_start = filename.find("hidden_") + len("hidden_")
    level_end = filename.find("error")
    level = int(filename[level_start:level_end].replace("_", ""))

    one_file = pd.read_csv(f'outputs/brute_force_2_with_frequency/{filename}', names = ["mse"])
    one_file['word'] = words['word']
    one_file['model_id'] = ID
    one_file['hidden'] = level
    one_file['train'] = samples[:,ID]
    one_file['test'] = tests[:,ID]
    one_file['holdout'] = holdouts[:,ID]
    dfs.append(one_file)

df = pd.concat(dfs, ignore_index=True)

# this function needs to change if you are calculating accuracy from featurewise accuracy rather than from mse:
df['accuracy'] = df['mse'].apply(lambda x: 0 if x > 0 else 1)

Save file for LMEMs

In [8]:
df.columns

Index(['mse', 'word', 'model_id', 'hidden', 'train', 'test', 'holdout',
       'accuracy'],
      dtype='object')

### If identifying a subset

In [12]:
df = df[df['test'] == False]
df = df[['model_id', 'hidden', 'word', 'mse', 'train', 'holdout']]

# if you have generated a subset:
df.to_csv('outputs/brute_force_2_with_frequency/subset_data_for_lmem_v2.csv', index=False)


## Data grouped by model or grouped by word

In [32]:
df_holdout = df[df['holdout']==True]

data_by_model_for_lmem = df_holdout.groupby(['model_id', 'hidden']).agg(
    mse_mean=('mse', 'mean'),
    mse_sd=('mse', 'std'),
    count=('mse', 'size')
).reset_index()

df_train = df[df['train']==True]

data_by_word_for_lmem = df_train.groupby(['word', 'hidden']).agg(
    mse_mean=('mse', 'mean'),
    mse_sd=('mse', 'std'),
    count=('mse', 'size')
).reset_index()

In [36]:
data_by_model_for_lmem.to_csv('~/Desktop/data_by_model_for_lmem.csv', index=False)
data_by_word_for_lmem.to_csv('~/Desktop/data_by_word_for_lmem.csv', index=False)

# Tabulate summary data per model
Using the "...summary.csv" files, tabulate the summary data for every model.

In [11]:
results = []
cols_in = ['hidden_units', 'accuracy_train', 'mse_train', 'accuracy_test', 'mse_test', 'accuracy_holdout', 'mse_holdout']
cols_out = ['model_id', 'hidden_units', 'accuracy_train', 'mse_train', 'accuracy_test', 'mse_test', 'accuracy_holdout', 'mse_holdout']

for filename in os.listdir(directory):
    if filename.endswith("summary.csv"):
        # Extract the ID and level from the filename
        id_start = filename.find("sample_") + len("sample_")
        id_end = filename.find("hidden")
        ID = int(filename[id_start:id_end].replace("_", ""))

        # Read the csv file
        summary = pd.read_csv(os.path.join(directory, filename))[cols_in]
        summary['model_id'] = ID

        # Save the contents
        results.append(summary)
        

pd.concat(results).reindex(columns=cols_out).to_csv('outputs/brute_force_2_with_frequency/model_summaries.csv', index = False)

